In [4]:
import torch
import torch.distributed as dist
import torch.multiprocessing as mp
from transformers import pipeline
from transformers.pipelines.pt_utils import KeyDataset
import datasets
from transformers import (
    AutoConfig,
    AutoModelForSeq2SeqLM,
    AutoTokenizer,
    DataCollatorForSeq2Seq,
    HfArgumentParser,
    M2M100Tokenizer,
    MBart50Tokenizer,
    MBart50TokenizerFast,
    MBartTokenizer,
    MBartTokenizerFast,
    Seq2SeqTrainer,
    Seq2SeqTrainingArguments,
    default_data_collator,
    set_seed,
)
from transformers.trainer_utils import get_last_checkpoint
from transformers.utils import check_min_version, send_example_telemetry
from transformers.utils.versions import require_version
import torch.nn.functional as F
from datasets import load_dataset
import re
import csv
from tqdm import tqdm

In [5]:
path_to_model = "" #预训练模型路径
path_to_train_dataset = "" #训练数据即路径
path_to_output = ""#输出路径 模型训练完保存位置

In [6]:
generation_config = dict(
    max_length=4096,
    min_length=None,
    do_sample=False,
    early_stopping=True,
    num_beams=1,
    temperature=15.0,
    top_k=None,
    top_p=None,
    length_penalty=1.0,
    num_return_sequences=3,
    max_time=None,
    num_beam_groups=1,
    output_scores=False,
)

rank = 0
model_path = path_to_model

_tokenizer = AutoTokenizer.from_pretrained(model_path)#加载预训练讽刺其
_model = AutoModelForSeq2SeqLM.from_pretrained(model_path)
pipe = pipeline("translation", model = _model, config = generation_config, batch_size=1, tokenizer = _tokenizer, max_length = 4096)

# create a dataloader
dataset = load_dataset("csv", data_files=path_to_train_dataset)
with open(path_to_output, "w") as f:
        writer = csv.writer(f)
        for out in tqdm(pipe(KeyDataset(dataset['train'], "source"), batch_size=8, truncation="only_first")):
            writer.writerow(out[0]["translation_text"])


OSError: Incorrect path_or_model_id: ''. Please provide either the path to a local folder or the repo_id of a model on the Hub.